In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
#from skimage.metrics import structural_similarity as ssim

import cv2
import numpy as np
import math
import os
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageFilter
from model import SRCNN


In [2]:
# Define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
    # Convert images to torch tensors
    target_data = torch.tensor(target, dtype=torch.float)
    ref_data = torch.tensor(ref, dtype=torch.float)
    
    # Calculate the squared difference between the two images
    diff = ref_data - target_data
    mse = torch.mean(diff ** 2)
    
    # Calculate PSNR
    psnr_value = 20 * torch.log10(255. / torch.sqrt(mse))
    
    return psnr_value.item()

# Define function for mean squared error (MSE)
def mse(target, ref):
    # Convert images to torch tensors
    target_data = torch.tensor(target, dtype=torch.float)
    ref_data = torch.tensor(ref, dtype=torch.float)
    
    # Calculate the squared difference between the two images
    diff = ref_data - target_data
    mse_value = torch.mean(diff ** 2)
    
    return mse_value.item()

# Define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    
    return scores


#### Super Resolution Convolution Neural Network (SRCNN) Model
1) Explain the achitecture and hyper parameters of the SRCNN network from the original paper
2) 

In [5]:
# Instantiate the model
model = SRCNN()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0003)
# Define the loss function
criterion = nn.MSELoss()

#### Next approach 